In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Output, Input
import math

In [ ]:
df = pd.read_csv(r'D:\GitHub\Cursos-EBAC\analiseDados\VisualizarComunicarDados\visualizacaoDadosAvancada\projeto\ecommerce_estatistica (1).csv')

In [ ]:
df['nota_arredondada'] = df['Nota'].apply(lambda x: 5 if x > 4.49 else math.trunc(x))

In [ ]:
lista_temporada = df['Temporada'].unique()
options = [{'label': temporada, 'value': temporada} for temporada in lista_temporada]

In [ ]:
def cria_graficos(selecao_checklist): 
    filtro_df = df[df['Temporada'].isin(selecao_checklist)] 
    df_group = ( filtro_df .groupby(['Temporada', 'nota_arredondada'], as_index=False)['N_Avaliações'] .sum() ) 
    fig1 = px.bar(df_group, x='nota_arredondada', y='N_Avaliações', color='Temporada', barmode='group', color_discrete_sequence= px.colors.sequential.RdBu) 
    fig1.update_layout( 
        title='Grafico Nota vs Numero de avaliações e Desconto', 
        xaxis_title='Notas',
        yaxis_title='Numero de avaliações', 
        legend_title='Temporada', 
        plot_bgcolor='rgba(222, 255, 253, 1)', 
        paper_bgcolor='rgba(186, 245, 241, 1)' ) 
    
    fig2 = px.line(df_group, x='nota_arredondada', y='N_Avaliações', color='Temporada')
    fig2.update_layout(
        xaxis_title='Nota',
        yaxis_title='Avaliações'
    )
    return fig1, fig2

In [ ]:
def cria_app():
    app = Dash(__name__)
    app.layout = html.Div([
        html.H1('Projeto Dashboard Interativo'),
        html.P('Esse Dashboard ira apresentar dois graficos usados para visualização avançada e diferentes tipos de interações.'),
        html.Br(),
        html.H2('Grafico de Barras'),
        dcc.Checklist(
            id = 'id_checklist_bar',
            options = options,
            value = [lista_temporada[0]]
        ),
        dcc.Graph(id='id_grafico_barras'),
        html.Br(),
        html.H2('Grafico de Linhas'),
        dcc.Graph(id='id_grafico_linhas')
    ])
    return app

In [ ]:
if __name__ == '__main__':
    app = cria_app()

    @app.callback(
        Output('id_grafico_barras', 'figure'),
        Output('id_grafico_linhas', 'figure'),
        Input('id_checklist_bar', 'value')
    )
    def atualiza_grafico(selecao_checklist):
        fig1, fig2= cria_graficos(selecao_checklist)
        return fig1, fig2

    app.run(debug=True, port=8050)